### Дилемма смещения и разброса. Полиномиальные признаки. Регуляризация

In [1]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики
%matplotlib inline
plt.style.use('seaborn-v0_8')

column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston_data = pd.read_csv('data/housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_data.head()

#Составляем список факторов (исключили целевой столбец)
features = boston_data.drop('MEDV', axis=1).columns
#Составляем матрицу наблюдений X и вектор ответов y
X = boston_data[features]
y = boston_data['MEDV']

Разделим дата-сет на тренировочную и тестовую выборки в соотношении 70/30

In [4]:
# инициализируем функцию по разделению дата сета на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split #as tts

# Делим выборку 70/30 и установим рандом - 40
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

# вернем результирующие размеры таблиц
print('Train:', X_train.shape, y_train.shape)
print('Train:', X_test.shape, y_test.shape)

Train: (354, 13) (354,)
Train: (152, 13) (152,)
